In [ ]:
import sys
sys.path.insert(0,'c:/MyDocs/integrated/') # adjust to your setup

%run "catalog_support.py" 

In [ ]:

alldf = pd.read_parquet(os.path.join(hndl.sandbox_dir,'state.parquet'))
alldf.date = pd.to_datetime(alldf.date)
statename = alldf.bgStateName.iloc[0]
state_file_handle = statename.replace(' ','_')

hide_blog_links = ['idaho','indiana','illinois','nevada', 'nebraska','michigan',
                   'kentucky','mississippi','alabama']
# print(alldf.columns)

In [ ]:
# print( th.getMoleculeImg(cas,size=300))
showHeader(statename.title(), link_up_level=1)

In [ ]:
# state abbreviations
s = """
ALABAMA	AL
ALASKA	AK
ARIZONA	AZ
ARKANSAS	AR
CALIFORNIA	CA
COLORADO	CO
CONNECTICUT	CT
DELAWARE	DE
FLORIDA	FL
GEORGIA	GA
HAWAII	HI
IDAHO	ID
ILLINOIS	IL
INDIANA	IN
IOWA	IA
KANSAS	KS
KENTUCKY	KY
LOUISIANA	LA
MAINE	ME
MARYLAND	MD
MASSACHUSETTS	MA
MICHIGAN	MI
MINNESOTA	MN
MISSISSIPPI	MS
MISSOURI	MO
MONTANA	MT
NEBRASKA	NE
NEVADA	NV
NEW HAMPSHIRE	NH
NEW JERSEY	NJ
NEW MEXICO	NM
NEW YORK	NY
NORTH CAROLINA	NC
NORTH DAKOTA	ND
OHIO	OH
OKLAHOMA	OK
OREGON	OR
PENNSYLVANIA	PA
RHODE ISLAND	RI
SOUTH CAROLINA	SC
SOUTH DAKOTA	SD
TENNESSEE	TN
TEXAS	TX
UTAH	UT
VERMONT	VT
VIRGINIA	VA
WASHINGTON	WA
WEST VIRGINIA	WV
WISCONSIN	WI
WYOMING	WY
MEXICO_MX
CANADA_CN

"""
s = s.replace('\t','_')
state_abbv_dic = {}
for line in s.split('\n'):
    tup = tuple(line.split('_'))
    try:
        state_abbv_dic[tup[0].lower()] = tup[1]
    except:
        pass

my_state_abbv = state_abbv_dic[statename]
# print(my_state_abbv)

In [ ]:
# get watershed df for determining which sheds to plot
#!!!! Need to move this to the caller! don't run with every watershed!
import geopandas as gpd
# gdb_path =  r"C:\MyDocs\integrated\ext_data\WBD_National_GDB.zip"
# huc8_layer_name = 'WBDHU8'  # Example layer name, use the one you found
#!!!!!

# Read the specific layer into a GeoDataFrame
# huc8_gdf = gpd.read_file(gdb_path, layer=huc8_layer_name)
# huc8_gdf = huc8_gdf.drop('loaddate',axis=1)
# huc8_gdf.to_parquet(os.path.join(hndl.sandbox_dir,'huc8_gdf.parquet'))

huc8_gdf = gpd.read_parquet(os.path.join(hndl.sandbox_dir,'huc8_gdf.parquet'))
huc8_gdf['huc8_name'] = huc8_gdf['name']
# print(huc8_gdf.columns)

In [ ]:
def xlate_val(n):
    if n==0:
        return ''
    if n<1000:
        return round_sig(n,1)
    x = round_sig(n,1)
    return x[0]+ 'k'

def make_annot(gb):
    annot = gb.copy()
    annot.DisclosureId = annot.DisclosureId.map(lambda x: xlate_val(x))
    #print(annot)
    piv = annot.pivot(index='County',columns='year',values='DisclosureId')
    piv.fillna('',inplace=True)
    #print(piv)
    return piv

def get_state_center(state):
    t = pd.read_csv(r"C:\MyDocs\OpenFF\src\openFF-catalog\work\state_coords.csv",
                   dtype={'Latitude':'float', 'Longitude':'float'})
    t = t[t.state==state]
    #print(t)
    return [t.Latitude.mean(),t.Longitude.mean()*-1]

# mapping code - much lifted from core.mapping
import pandas as pd
import numpy as np
import geopandas as gpd
from shapely.geometry import Point
import openFF.common.text_handlers as th

final_crs = 4326 # WGS84
proj_crs = 3857 # convert to this when calculating distances
def_buffer = 1609.34 # one mile

def prep_shape(statename='ohio'):
    fn = r"C:\MyDocs\integrated\ext_data\georef-united-states-of-america-state.geojson"
    geojson = gpd.read_file(fn)
    geojson['StateName'] = geojson.ste_name.str.lower()
    # print(geojson.StateName)
    # geojson['CountyName'] = geojson.coty_name.str.lower()
    # geojson = mapping.fix_county_names(geojson)
    geojson = geojson[(geojson.StateName==statename)]
    return geojson
    
def CountyMap(df):
    start_loc = get_state_center(statename)
    #print(statename,start_loc)
    cond = ~df.location_error
    if cond.sum()==0:  # no valid fracks for this state
        display(md(f'### No mappable fracks for {statename}!'))
        display(md(f'This is usually caused when "CountyName" in FracFocus dont match standard names'))
        return
    gb = df[cond].groupby(['bgStateName','bgCountyName',
                                                   'DisclosureId'],as_index=False).size()
    gb = gb.groupby(['bgStateName','bgCountyName'],as_index=False)['DisclosureId'].count().rename({'bgStateName':'StateName',
                                                                                                'bgCountyName':'CountyName',
                                                                                                'DisclosureId':'value'},
                                                                                                axis=1)
    zoom = 6
    if statename in ['texas','california']:
        zoom = 5
    if statename in ['alaska']:
        zoom = 4

    geojson = prep_shape(statename)

    fig = mapping.create_county_choropleth(gb,include_shape=True,area_df=geojson,
                             plotlog=True,custom_scale= [0,1,2,3,4,5],
                             start_loc=start_loc, # center of state's data
                             legend_name='Number of FracFocus disclosures',
                             start_zoom=zoom,fields=['StateName','CountyName','orig_value','county_link'],
                             aliases=['State: ','County: ','Number Fracking disclosures: ','Link to county page: '])
    
    fn = os.path.join(hndl.browser_states_dir,state_file_handle,'county_choropleth.html')
    fig.save(fn)

def CountyCntTable(df):
    # first, make the general searchable table
    gb = df.groupby(['bgCountyName','DisclosureId'],as_index=False)['date'].first()
    gb['year'] = gb.date.dt.year.astype('str')
    gb1 = gb.groupby(['bgCountyName'],as_index=False)['DisclosureId'].count().rename({'DisclosureId':'disclosure_count'},
                                                                                  axis=1)
    gb2 = gb1.copy()
    gbop = df.groupby('bgCountyName')['OperatorName'].agg(lambda x:x.value_counts().index[0:4]).reset_index()
    gbop.OperatorName = gbop.OperatorName.map(lambda x: th.xlate_to_str(x,'; ',sort=False))
    gbop = gbop.rename({'OperatorName':'Top Operators'},axis=1)
    gb2 = pd.merge(gb2,gbop,on='bgCountyName',how='left')

#     gbprop = df[df.bgCAS=='proprietary'].groupby('bgCountyName',as_index=False).size()
# #     gbprop.bgCAS.fillna(0,inplace=True)
#     gbprop = gbprop.rename({'size':'Trade Secret records'},axis=1)
#     gb2 = pd.merge(gb2,gbprop,on='bgCountyName',how='left')
#     gb2['Trade Secret records'].fillna(0,inplace=True)
    
    gbtbwv = df.groupby(['bgCountyName','DisclosureId'],as_index=False)['TotalBaseWaterVolume'].first()
    gbtbwv = gbtbwv.groupby('bgCountyName',as_index=False)['TotalBaseWaterVolume'].sum().rename({'TotalBaseWaterVolume':'tot_gallons_water'},
                                                                                                axis=1)
    gbtbwv.tot_gallons_water = gbtbwv.tot_gallons_water.map(lambda x: th.round_sig(x,3))
    gb2 = pd.merge(gb2,gbtbwv,on='bgCountyName',how='left')

    #print(gb2.head())
    gb2['County'] = '<center><h4>'+gb2.bgCountyName.str.title().map(lambda x: th.getCountyLink(x,statename,x))+'</h4></center>'
    gb2 = gb2.drop('bgCountyName',axis=1)
    iShow(gb2.sort_values('disclosure_count',ascending=False)[['County','disclosure_count',
                                                               #'Trade Secret records'
                                                               'tot_gallons_water',
                                                               'Top Operators']].reset_index(drop=True),
         classes="display compact cell-border")

def fetch_watersheds_for_state(state_abbv=my_state_abbv):
    c1 = huc8_gdf.states.str.contains(state_abbv)
    out = huc8_gdf[c1]
    # print(f'Of {len(huc8_gdf)} watersheds, returning {len(huc8_gdf[c1])}')
    return out

def WatershedMap(df):
    start_loc = get_state_center(statename)

    fn = r"C:\MyDocs\OpenFF\data\non-FF\georef-united-states-of-america-state.geojson"
    statejson = gpd.read_file(fn)
    statejson['StateName'] = statejson.ste_name.str.lower()
    # geojson['CountyName'] = geojson.coty_name.str.lower()
    # geojson = mapping.fix_county_names(geojson)
    # geojson = geojson[(geojson.StateName==statename)&(geojson.CountyName==countyname)]
    statejson = statejson[statejson.StateName==statename]
    
    #print(statename,start_loc)
    # cond = ~df.location_error
    # if cond.sum()==0:  # no valid fracks for this state
    #     display(md(f'### No mappable fracks for {statename}!'))
    #     display(md(f'This is usually caused when "CountyName" in FracFocus dont match standard names'))
    #     return
    gb = df.groupby(['huc8','DisclosureId'],as_index=False).size()
    gb = gb.groupby(['huc8'],as_index=False).size().rename({'size':'value'},axis=1)
    gb['statename'] = statename
    zoom = 6
    if statename in ['texas','california']:
        zoom = 5
    if statename in ['alaska']:
        zoom = 4

    geojson = fetch_watersheds_for_state()
    # print(geojson.columns)
    fig = mapping.create_watershed_choropleth(gb,include_shape=True,area_df=geojson,
                                              include_state=True,state_df=statejson,
                             plotlog=True,custom_scale= [0,1,2,3,4,5],
                             start_loc=start_loc, # center of state's data
                             legend_name='Number of FracFocus disclosures',
                             start_zoom=zoom,fields=['huc8_name','huc8','orig_value','watershed_link'],
                             aliases=['Watershed name: ','Watershed code: ','Number Fracking disclosures: ','Link to watershed page: ']
                                          )
    
    # fn = os.path.join(hndl.browser_states_dir,state_file_handle,'county_choropleth.html')
    # fig.save(fn)

# states_sheds = fetch_watersheds_for_state()


---
# Links and Resources

In [ ]:
sn = statename.replace(' ','-')
s = ''
if not statename in hide_blog_links:
    s+= f"> **[Open-FF summary stats and posts about {statename.title()}](https://open-ff.org/{sn}-fracfocus/)**<br>"
s+= f"**[FracTracker Alliance maps of {statename.title()}](https://www.fractracker.org/map/us/{sn}/)**"
display(md(s))

# Where are the fracking locations in this state?
This is not an exhaustive set of wells in these counties; it is only those wells for which the operating company submits a chemical disclosure to FracFocus.  In addition, this map omits disclosures for which location information is conflicting, such as the Latitude/Longitude values are outside of the reported county.

### by county

In [ ]:
CountyMap(alldf)

### by watershed

In [ ]:
WatershedMap(alldf)

---
## Number of disclosures per month


In [ ]:
gb = alldf.groupby('DisclosureId',as_index=False)[['date','no_chem_recs']].first()
gb = gb[gb.date.dt.year>2010]
# gb1 = alldf.groupby('DisclosureId',as_index=False)['ingKeyPresent'].sum()
# mg = pd.merge(gb,gb1,on='DisclosureId',how='left')
gb2 = gb[~gb.no_chem_recs].groupby('date').size()
allwk_sum = gb2.resample("M").sum()
ax = allwk_sum.plot(figsize=(7,5), ylabel='Number of disclosures',label='with chemical records');
ax.set_title(f'Monthly Number of {statename.title()} Disclosures',fontsize=15);


gb3 = gb[gb.no_chem_recs].groupby('date').size()
# alldfv1 = master_df[~master_df.ingKeyPresent].groupby('DisclosureId',as_index=False)[['date','TotalBaseWaterVolume']].first()
# gbv1 = gb3.groupby('date').size()
allwk_sumv1 = gb3.resample("m").sum()
allwk_sumv1.plot(ax=ax,label='without chemical records');
ax.legend();
plt.tight_layout()
plt.savefig(os.path.join(hndl.browser_states_dir,state_file_handle,'number_disc_in_state.jpg'))


---
## Water use
### Gallons used, recorded as TotalBaseWaterVolume

In [ ]:
wdf = alldf.groupby('DisclosureId',as_index=False)[['date','TotalBaseWaterVolume','APINumber','bgStateName']].first()
wdf = wdf[wdf.date.dt.year>2010]
gb1 = wdf.groupby('date')['TotalBaseWaterVolume'].median()
# gb1 = gb1/1000000
allwk_tbwv = gb1.resample("M").median()
# don't show the final two months because it is often wonky because it is not full data
ax = allwk_tbwv.iloc[:-2].plot(figsize=(7,5), ylabel='Median gallons of water\n each month',style='o',xlabel='');
ax.set_title(f'Typical quantity of water used in {statename.title()}',fontsize=14);
ax.tick_params(axis="y", labelsize=13)
ax.tick_params(axis="x", labelsize=13)
plt.ylabel('Median water use per month\n (gallons)',fontsize=13);
ax = gca().yaxis.set_major_formatter(mpl.ticker.StrMethodFormatter('{x:,.0f}'))
plt.tight_layout()
# plt.savefig(os.path.join(hndl.blog_im_dir,'median_water_used.jpg'),dpi=150)
plt.savefig(os.path.join(hndl.browser_states_dir,state_file_handle,'water_use_in_state.jpg'))

In [ ]:
# see if we should build water source plot
gb = alldf.groupby('DisclosureId',as_index=False)[['TotalBaseWaterVolume','ws_perc_total','bgStateName',
                                                   'perc_pw','perc_gw_high_TDS','perc_gw_low_TDS','date',
                                                   'perc_sw_high_TDS','perc_sw_low_TDS',
                                                   'perc_other_high_TDS','perc_other_low_TDS']].first()


ws_start = datetime.datetime(year=2023,month=12,day=1)
c1 = gb.ws_perc_total>0
c2 = gb.ws_perc_total==100
c3 = gb.date>=ws_start
num_ws_disc = len(gb[c1])

# all_disc = len(gb)
all_disc_since_start = len(gb[c3])
frac_ws_disc = num_ws_disc/all_disc_since_start


if num_ws_disc >5:
    display(md(f'### Reported (optional) water source'))
    display(md("""In late 2023, [FracFocus added an option](https://fracfocus.org/news/water-source-data) for companies to report the type of water source(s)
    they used in the fracking job. Because this reporting is
    optional, this figure represent only a fraction of the disclosure published since its introduction.
    The categories available are: """))
    display(md(""">##### Ground water, Surface water, Produced water, and Other."""))
    display(md("""Additionally, ground, surface and other are categorized by low or high "total dissolved solids" (TDS).  We don't display those distinctions here. """))
    
    gb['perc_gw'] = gb.perc_gw_low_TDS + gb.perc_gw_high_TDS  
    gb['perc_sw'] = gb.perc_sw_low_TDS + gb.perc_sw_high_TDS  
    gb['perc_other'] = gb.perc_other_low_TDS + gb.perc_other_high_TDS  

    gb['vol_gw'] = gb.perc_gw * gb.TotalBaseWaterVolume /100
    gb['vol_sw'] = gb.perc_sw * gb.TotalBaseWaterVolume /100
    gb['vol_pw'] = gb.perc_pw * gb.TotalBaseWaterVolume /100
    gb['vol_other'] = gb.perc_other * gb.TotalBaseWaterVolume /100    
    num_disc = len(gb[c1])

    out = gb.groupby('bgStateName',as_index=False)[['vol_gw','vol_sw','vol_pw','vol_other']].sum()
    out = out.drop('bgStateName',axis=1)
    meltdf = out.melt()
    ax = meltdf.plot(kind='bar',legend=False)
    plt.suptitle(f'Reported water source for {num_disc} {statename.title()} fracking jobs')
    plt.title(f'(data from about {round(frac_ws_disc,3):.0%} of disclosures since Dec. 2023)',fontsize=10)
    plt.xlabel('Reported water source')
    plt.ylabel('Total Gallons')
    ax.set_xticklabels(['Ground\nWater','Surface\nWater','Produced\nWater','Other'])
    plt.xticks(rotation=0)  # Rotate x-axis labels if needed
    ax = gca().yaxis.set_major_formatter(mpl.ticker.StrMethodFormatter('{x:,.0f}'));
    plt.tight_layout()
    plt.savefig(os.path.join(hndl.browser_states_dir,state_file_handle,'water_source_in_state.jpg'))    
    # Show the plot
    plt.show()


<a id='counties'></a>
---
## County-based details

Click on the county name in the table to view a county summary page.

In [ ]:
CountyCntTable(alldf)

In [ ]:
display(md("--- \n # Trade Secret designations"))
testtitle = statename.title() +': Trade Secret frequency'
# print(alldf.columns)
statelab = statename.replace(' ','_')
outfn = os.path.join(hndl.browser_states_dir,statelab,'state_proprietary_plot.jpg')
c_plots.proprietary_bars(alldf,testtitle,save_file=outfn)

<a id='operators'></a>
---
## Who are the Operators in this state?

In [ ]:
ogb = alldf.groupby(['bgOperatorName','DisclosureId'],as_index=False).size()
ogbname = alldf.groupby(['bgOperatorName'])['OperatorName'].agg(lambda x: x.value_counts().index[0])
ogbname = ogbname.reset_index()
mg = pd.merge(ogb,ogbname,on='bgOperatorName',how='left')

In [ ]:
ax = mg.OperatorName.value_counts()[:15].plot.barh(figsize=(8,5))
ax.set_title(f'Top Operators in {statename.title()}')
ax.set_xlabel('Number of disclosures in FracFocus');
plt.tight_layout();
plt.savefig(os.path.join(hndl.browser_states_dir,state_file_handle,'state_major_operators.jpg'))

In [ ]:
def make_water(row):
    s = str(th.round_sig(row.TotalBaseWaterVolume,3))
    s += '<br>'
    s += str(th.round_sig(row.TBWV90,3))
    return s

alldf['year'] = alldf.date.dt.year.astype(str)
gbOp = alldf.groupby(['DisclosureId','bgCountyName','bgOperatorName','year'],as_index=False)['TotalBaseWaterVolume'].first()
gbOp.bgCountyName = gbOp.bgCountyName.str.title()

gbOp1 = gbOp.groupby('bgOperatorName',as_index=False)['DisclosureId'].count().rename({'DisclosureId':'num_fracks'},axis=1)

gbOp2 = gbOp.groupby(['bgOperatorName','year'],as_index=False)['DisclosureId'].count()
gbOp2['year_cnt'] = gbOp2.year + '(' + gbOp2.DisclosureId.astype(str) + ')'

gbOpY = gbOp2.groupby('bgOperatorName')['year_cnt'].apply(set).reset_index()
gbOpY['years'] = gbOpY.year_cnt.map(lambda x: th.xlate_to_str(x,sep='<br>'))

gbOp3 = gbOp.groupby(['bgOperatorName','bgCountyName'],as_index=False)['DisclosureId'].count()
gbOp3['counties_cnt'] = gbOp3.bgCountyName + '(' + gbOp3.DisclosureId.astype(str) + ')'

gbOp4 = gbOp3.groupby('bgOperatorName')['counties_cnt'].apply(set).reset_index()
gbOp4['counties'] = gbOp4.counties_cnt.map(lambda x: th.xlate_to_str(x,sep='<br>'))

# gbOp5 = df.groupby('bgOperatorName')['OperatorName'].agg(lambda x: x.value_counts().index[0])
gbOp5 = alldf.groupby('bgOperatorName')['OperatorName'].apply(set).reset_index()
gbOp5['names'] = gbOp5.OperatorName.map(lambda x: th.xlate_to_str(x,sep='<br>'))
# gbOp5.names = gbOp5.names + '<br>[' + gbOp5.bgOperatorName + ']'

gbOp6 = gbOp.groupby('bgOperatorName',as_index=False)['TotalBaseWaterVolume'].median()
gbOp6.rename({'TotalBaseWaterVolume':'Water,\nmedian (gal)'},axis=1,inplace=True)
# gbOp7 = gbOp.groupby('bgOperatorName',as_index=False)['TotalBaseWaterVolume'].agg(lambda x: np.percentile(x,90))
gbOp7 = gbOp.groupby('bgOperatorName',as_index=False)['TotalBaseWaterVolume'].max()
gbOp7.rename({'TotalBaseWaterVolume':'Water,\nmax (gal)'},axis=1,inplace=True)
mg = pd.merge(gbOp6,gbOp7,on='bgOperatorName')
# mg.fillna(0,inplace=True)
# mg['TBWV'] = mg.apply(lambda x: make_water(x),axis=1)
mg = pd.merge(mg,gbOp1,on='bgOperatorName')
mg = pd.merge(mg,gbOpY,on='bgOperatorName')
mg = pd.merge(mg,gbOp4[['bgOperatorName','counties']],on='bgOperatorName')
mg = pd.merge(mg,gbOp5,on='bgOperatorName').sort_values('num_fracks',ascending=False)

mg['link'] = mg.bgOperatorName.map(lambda x: th.getOpLink(x,x,up_level=True))
# mg.names = mg.names + '<br>[' + mg.link + ']'
mg.names = '<center><h3>'+mg.names+'<br><br>'+mg.link+'</h3></center>'

iShow(mg[['names','num_fracks','years','counties','Water,\nmedian (gal)','Water,\nmax (gal)']].reset_index(drop=True),
      maxBytes=0,columnDefs=[{"width": "150px", "targets": 0}])

<a id='duplicates'></a>
# Duplicate records
As of Aug. 2024, the disclosures of many operator companies had **[apparently unintentional duplicate records](https://open-ff.org/2024/09/17/fracfocus-duplicate-records/)** in them.  Such duplicates can distort calculations of chemical quantity.  In this section, we compare the current status of those records **for this state** with the baseline taken in Aug. 2024.

In [ ]:
import os
fn = os.path.join(r"G:\My Drive\webshare\daily_status",'dup_rec_baseline.parquet')
baseline = pd.read_parquet(fn)
bl = baseline[baseline.bgStateName==statename].copy()
# print(bl.columns)
blapis = bl.APINumber.unique().tolist()

if len(bl)==0:
    display(md('## No duplicates found in baseline (August 2024)'))
else:
    display(md('### Baseline summary (Aug. 2024)'))
    gb = bl.groupby(['bgStateName'],as_index=False).size().rename({'size':'num disclosures'},axis=1)
    gb1 = bl.groupby(['bgStateName'],as_index=False).mass.sum()
    mg = pd.merge(gb,gb1,on = 'bgStateName',how='left')
    mg['mean mass per disclosure'] = round(mg.mass/mg['num disclosures'])
    mg = mg.rename({'mass':'mass of all duplicates (pounds)'},axis=1)
    display(mg)

In [ ]:
unfiltdf = pd.read_parquet(os.path.join(hndl.sandbox_dir,'state_unfilt.parquet'))
c = unfiltdf.dup_rec
new = unfiltdf[c].copy()
if len(new)==0:
    display(md('## No duplicates found in current data set'))
else:
    display(md('---\n ### Current summary'))
    # look for removed apis
    newapis = new.APINumber.unique().tolist()
    allapis = alldf.APINumber.unique().tolist()
    removed = []
    corrected = []
    for api in blapis:
        if not api in newapis:
            if api in allapis:
                corrected.append(api)
            else:
                removed.append(api)
    display(md(f'Baseline disclosures with duplicates **REMOVED** from FracFocus: {len(removed)}'))   
    display(md(f'Baseline disclosures with duplicates **CORRECTED** in FracFocus: {len(corrected)}'))   

    gb = new.groupby(['bgStateName','DisclosureId'],as_index=False).size()
    gb1 = new.groupby(['bgStateName','DisclosureId'],as_index=False).mass.sum()
    mg = pd.merge(gb,gb1,on=['bgStateName','DisclosureId'], how='left')

    sgb = mg.groupby(['bgStateName'],as_index=False).size().rename({'size':'num disclosures'},axis=1)
    sgb1 = mg.groupby(['bgStateName'],as_index=False).mass.sum()
    smg = pd.merge(sgb,sgb1,on = 'bgStateName',how='left')
    smg['mean mass per disclosure'] = round(smg.mass/smg['num disclosures'])
    smg = smg.rename({'mass':'mass of all duplicates (pounds)'},axis=1)
    iShow(smg)
    display(md('---'))

In [ ]:
if len(new)>0:   
    display(md('---\n### List of disclosures with duplicates\n\n'))
    gbsize = new.groupby(['DisclosureId'],as_index=False).size().rename({'size':'numdups'},axis=1)
    gbmain = new.groupby(['DisclosureId'],as_index=False)[['OperatorName','APINumber','date',
                                                           'bgStateName','bgCountyName']].first()
    lmg = pd.merge(gbmain,gbsize,on='DisclosureId',how='left')
    lmg['FF_disc'] = lmg.apply(lambda x: th.getFFLink(x,fmt='short'),axis=1)
    lmg.bgStateName = lmg.bgStateName.str.title()
    lmg.bgCountyName = lmg.bgCountyName.str.title()
    out = lmg[['FF_disc','date','numdups','bgStateName','bgCountyName','OperatorName']]
    out.columns = ['API Number<br> (and link)','date','Number of<br>duplicate<br>records','State','County','Operator']
    display(out)
    

In [ ]:
# # wells_in_dist_fn = 'FFwells_in_school_districts.csv'
# distFF = pd.read_csv(wells_in_dist_fn)
# distFF = distFF[distFF.bgStateName==statename]
# distFF.num_FF_wells.fillna(0,inplace=True)
# iShow(distFF[['NAME','GEOID','num_FF_wells','num_all_wells']].reset_index(drop=True),
#      classes="display compact cell-border")